In [16]:
import numpy as np
import matplotlib.pyplot as plt


# Load data from the .txt file
#ssh -Y cic@148.204.66.123
#ssh -Y osicmicrose@148.204.66.53
#scp osicmicrose@148.204.66.53:/home/osicmicrose/Desktop/EDA/SNN_IPN/sim_results/tb_4x8x4_data.txt . 
#scp cic@148.204.66.123:/home/cic/.xschem/simulations/tb_4x8x4_data.txt  .
# data = np.loadtxt('tb_4x8x4_data.txt', skiprows=1)  # Adjust 'data.txt' to your filename


In [17]:
def read_ngspice_raw(file_path):
    # Initialize a list to store the data
    data = []

    with open(file_path, 'rb') as file:  # Open in binary mode
        content = file.read()  # Read the entire content
        # Try decoding with a fallback encoding
        try:
            content = content.decode('utf-8')
        except UnicodeDecodeError:
            content = content.decode('latin1')   # Fallback to a different encoding

    # Process the lines
    lines = content.splitlines()
    lines = lines[29:]
    for line in lines:
        # Ignore comment lines and empty lines
        if line.startswith('#') or line.strip() == '':
            continue
        # Split the line into columns and convert to float
        row = list(map(float, line.split()))
        data.append(row)

    # Convert the list of rows into a numpy array (matrix)
    matrix = np.array(data)
    return matrix

data = read_ngspice_raw("tb_4x8x4_data.raw")

ValueError: could not convert string to float: '\x95Ö&è'

In [ ]:


# Store each column into separate vectors
time = data[:, 0]
vin = data[:, 1]
Vr1 = data[:, 2]
iVread = data[:, 3]
hx = data[:, 4]
x = data[:, 5]
n = data[:, 6:10]
m = data[:, 10:14]
j = data[:, 14:22]
ij = data[:, 22:54]
jk = data[:, 54:]

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,2))
for i in range(n.shape[1]):
    ax.plot(time*1000, n[:,i]+(1.8*i), label = f"N{i+1}")
ax.legend()
ax.grid()
ax.set_xlabel("Time [ms]")
ax.set_ylabel("Spikes [V]")
ax.set_title("Input Layer Neural activity")

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,2))
for i in range(j.shape[1]):
    ax.plot(time*1000, j[:,i]+(1.8*i), label = f"N{i+1}")
ax.legend()
ax.grid()
ax.set_xlabel("Time [ms]")
ax.set_ylabel("Spikes [V]")
ax.set_title("Hidden Layer Neural activity")

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,2))
for i in range(m.shape[1]):
    ax.plot(time*1000, m[:,i]+(1.8*i), label = f"M{i+1}")
ax.legend()
ax.grid()
ax.set_xlabel("Time [ms]")
ax.set_ylabel("Spikes [V]")
ax.set_title("Output Layer Neural activity")

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,2))
ax.plot(time*1000, iVread, label = "Vin")
ax.set_xlabel("Time [ms]")
ax.set_ylabel("Output Current [uA]")
ax.set_title("OutputCurrent")
ax.grid()

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,3))

ax.plot(time*1000, hx, label = "$\hat x$")
ax.plot(time*1000, x, label = "x")
# ax.plot(time*1000, Vr1, label = "Vr1")
ax.legend()
ax.grid()
ax.set_xlabel("Time [ms]")
ax.set_ylabel("Voltage [V]")
ax.set_title("Votlage signals")

In [ ]:
len(time)

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,3))

ax.plot(time*1000, x - hx, label = "error")
ax.legend()
ax.grid()
ax.set_xlabel("Time [ms]")
ax.set_ylabel("Voltage [V]")
ax.set_title("Votlage signals")

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,3))
for i in range(ij.shape[1]):
    ax.plot(time*1000, 5 - ij[:,i], label = f"ij{i+1}")
# ax.legend()
ax.grid()
ax.set_xlabel("Time [ms]")
ax.set_ylabel("ThicknessFill [V]")
ax.set_title("Memristance value ij")

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,3))
for i in range(jk.shape[1]):
    ax.plot(time*1000, 5 - jk[:,i], label = f"jk{i+1}")
# ax.legend()
ax.grid()
ax.set_xlabel("Time [ms]")
ax.set_ylabel("ThicknessFill [V]")
ax.set_title("Memristance value jk")